## Setup

In [13]:
data_dir = "../data/"
output_dir = "../output/"

In [14]:
import pandas as pd
df = pd.read_csv(data_dir + "taxonomy.csv", index_col = 0)
df.head()

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Abiotrophia_defectiva,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_fermentans,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000018,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_intestini,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00957,0.006907,0.0,0.0,0.0,0.0,0.009285
s__Actinomyces_graevenitzii,0.003491,0.000110,0.000004,0.000435,0.001297,0.000168,0.005996,0.000021,0.000039,0.000004,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Actinomyces_odontolyticus,0.000377,0.000005,0.000000,0.000014,0.000028,0.000107,0.000080,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000027


In [15]:
species = df.copy()
species[species < 0.00001] = 0
species.head()

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Abiotrophia_defectiva,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_fermentans,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000018,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Acidaminococcus_intestini,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00957,0.006907,0.0,0.0,0.0,0.0,0.009285
s__Actinomyces_graevenitzii,0.003491,0.00011,0.0,0.000435,0.001297,0.000168,0.005996,0.000021,0.000039,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
s__Actinomyces_odontolyticus,0.000377,0.00000,0.0,0.000014,0.000028,0.000107,0.000080,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000027


In [16]:
with open(output_dir + "MH_species.txt") as f:
    MH_species = f.read().splitlines()
with open(output_dir + "MN_species.txt") as f:
    MN_species = f.read().splitlines()
MH_species

['s__Alistipes_senegalensis',
 's__Bacteroidales_bacterium_ph8',
 's__Bifidobacterium_adolescentis',
 's__Bifidobacterium_angulatum',
 's__Bifidobacterium_catenulatum',
 's__Lachnospiraceae_bacterium_8_1_57FAA',
 's__Sutterella_wadsworthensis']

In [17]:
MH_species_metagenome = species[species.index.isin(MH_species)]
MN_species_metagenome = species[species.index.isin(MN_species)]
display(MH_species_metagenome)
display(MN_species_metagenome.head())

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Alistipes_senegalensis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.001058,0.000000,0.000000,0.000712,0.000000,0.000000,0.000000,0.000000
s__Bacteroidales_bacterium_ph8,0.000000,0.000103,0.000212,0.000000,0.000763,0.000000,0.00000,0.000000,0.000000,0.000050,...,0.000000,0.000000,0.009711,0.004391,0.005546,0.060104,0.000000,0.007427,0.000000,0.002521
s__Bifidobacterium_adolescentis,0.022290,0.227322,0.090593,0.091049,0.070562,0.020331,0.17816,0.011900,0.022083,0.136260,...,0.001126,0.001084,0.016068,0.004492,0.013137,0.000000,0.013176,0.010072,0.035911,0.045419
s__Bifidobacterium_angulatum,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
s__Bifidobacterium_catenulatum,0.004416,0.000228,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000490,0.000000,...,0.000394,0.000489,0.000000,0.000640,0.002786,0.000000,0.000245,0.000000,0.001325,0.009873
s__Lachnospiraceae_bacterium_8_1_57FAA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.001437,0.000000,0.000000,...,0.001264,0.000895,0.000000,0.000371,0.000345,0.001208,0.000000,0.000000,0.000000,0.001000
s__Sutterella_wadsworthensis,0.006551,0.000183,0.001943,0.007306,0.000000,0.000000,0.00000,0.002993,0.004440,0.000773,...,0.000000,0.000000,0.000000,0.005554,0.016498,0.007777,0.005217,0.000000,0.001308,0.001799


,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
Species,,,,,,,,,,,,,,,,,,,,,
s__Anaerotruncus_colihominis,0.000000,0.0,0.0,0.000000,0.000932,0.000000,0.000000,0.0,0.000125,0.0,...,0.000031,0.0,0.000016,0.000373,0.000233,0.00029,0.0,0.0,0.000173,0.000463
s__Atopobium_parvulum,0.000000,0.0,0.0,0.000075,0.000000,0.000000,0.000458,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000
s__Bifidobacterium_dentium,0.000016,0.0,0.0,0.000000,0.000000,0.000044,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000601,0.000000
s__Blautia_producta,0.000000,0.0,0.0,0.000000,0.000249,0.000000,0.000000,0.0,0.000350,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000
s__candidate_division_TM7_single_cell_isolate_TM7c,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000


In [20]:
import numpy as np

def shannon(df):
    shannoned = np.log(df) * df * -1
    filled = shannoned.fillna(0)
    sums = list(filled.sum(axis=0, ))
    columns = list(df.columns)
    frame = pd.DataFrame(pd.Series(data=sums, index=columns)).T
    frame.index = ['shannon']
    return frame

In [21]:
MH_shannon = shannon(MH_species_metagenome)
MN_shannon = shannon(MN_species_metagenome)
MH_shannon.iloc[:, :10]

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217
shannon,0.141669,0.341187,0.231469,0.254122,0.192554,0.079201,0.307339,0.079531,0.111983,0.277626


In [22]:
def richness(df):
    frame = pd.DataFrame((df > 0).sum()).T
    frame.index = ['Richness']
    return frame

In [23]:
R_MH = richness(MH_species_metagenome)
R_MN = richness(MN_species_metagenome)
R_MH.iloc[:, :10]

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217
Richness,3,4,3,2,2,1,1,3,3,3


In [24]:
MH_prime = 7
MN_prime = 31

In [26]:
psi_MH = pd.DataFrame((R_MH / MH_prime).values * MH_shannon.values, 
                      columns=MH_shannon.columns, index=["psi_MH"])
psi_MN = pd.DataFrame((R_MN / MN_prime).values * MN_shannon.values, 
                      columns=MN_shannon.columns, index=["psi_MN"])
psi_MN

,SAMD00036192,SAMD00036193,SAMD00036194,SAMD00036197,SAMD00036204,SAMD00036208,SAMD00036210,SAMD00036211,SAMD00036215,SAMD00036217,...,SAMN07534319,SAMN07534496,SAMN07534530,SAMN07534656,SAMN07534777,SAMN07534852,SAMN07534870,SAMN07534977,SAMN07534978,SAMN07535110
psi_MN,0.168244,0.009999,0.05035,0.03906,0.167363,0.056643,0.180919,0.159642,0.103282,0.0,...,0.00436,0.003063,0.027818,0.013443,0.00907,0.000184,0.004925,0.000188,0.018424,0.045176


In [27]:
GMHI = pd.DataFrame(np.log10((psi_MH+0.00001).values / (psi_MN+0.00001).values), 
                    columns=psi_MH.columns, index=["GMHI"]).T
GMHI

,GMHI
SAMD00036192,-0.442595
SAMD00036193,1.289568
SAMD00036194,0.294476
SAMD00036197,0.269188
SAMD00036204,-0.483122
...,...
SAMN07534852,2.811760
SAMN07534870,0.875898
SAMN07534977,2.077392
SAMN07534978,0.502971


In [28]:
(GMHI > 0).sum()

GMHI    2605
dtype: int64

In [29]:
isHealthy = pd.read_csv(data_dir + "isHealthy.csv", index_col=0)
isHealthy.T.head()

,isHealthy
SAMD00036192,True
SAMD00036193,True
SAMD00036194,False
SAMD00036197,True
SAMD00036204,True


In [30]:
def balanced_accuracy(GMHI, y):
    full = pd.concat([GMHI, y], axis=1)
    healthies = full[full['isHealthy']].iloc[:, [0]]
    predictedHealthy = int((healthies > 0).sum())
    unhealthies = full[~full['isHealthy']].iloc[:, [0]]
    predictedNot = int((unhealthies < 0).sum())
    P_H = predictedHealthy / len(healthies)
    P_N = predictedNot / len(unhealthies)
    chi = 0.5 * (P_H + P_N)
    return chi
balanced_accuracy(GMHI, isHealthy.T)

0.6968543052230989

## Balanced Accuracy of 0.697 hurray